#### 森羊双十一天猫数据分析报告

In [1]:
# %load prep.py
# %load prep.py

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'].insert(0, 'SimHei')
plt.rcParams['axes.unicode_minus'] = False


In [2]:
%config InlineBackend.figure_format = 'svg'

In [3]:
orders_df = pd.read_excel('res/order2021.xlsx', true_values=['是'], false_values=['否'])
orders_df

,orderID,userID,goodsID,price,num,orderAmount,payment,discount,orderTime,chargeback
0,sys-2021-129388536,user-233240,PR000366,838,1,838,838,NaN,2021-01-05 00:01:11,False
1,sys-2021-129388537,user-184666,PR000716,836,4,3344,3344,NaN,2021-01-05 00:02:12,True
2,sys-2021-129388538,user-283751,PR000323,1648,1,1648,1648,NaN,2021-01-05 00:02:46,True
3,sys-2021-129388539,user-279635,PR000652,651,1,651,651,NaN,2021-01-05 00:03:37,False
4,sys-2021-129388540,user-124652,PR000774,1415,2,2830,2830,NaN,2021-01-05 00:04:44,False
...,...,...,...,...,...,...,...,...,...,...
33496,sys-2021-129422913,user-235535,PR000164,593,4,2372,2372,,2021-01-13 23:57:24,True
33497,sys-2021-129422914,user-139042,PR000096,661,1,661,661,,2021-01-13 23:57:51,False
33498,sys-2021-129422915,user-133379,PR000984,696,1,696,696,,2021-01-13 23:58:19,False
33499,sys-2021-129422916,user-156013,PR000753,1484,4,5936,5936,,2021-01-13 23:59:24,False


In [4]:
orders_df['discount'] = orders_df.discount.replace(r'\s', '', regex=True).fillna('')

In [5]:
orders_df.shape

(33501, 10)

In [6]:
orders_df.discount.unique()

array(['', '3件8折', '2件9折', '1件95折', '满2700返180券', '满1800减80', '满600减30',
       '满4400返320券', '满3600返180券', '4件以上7折', '满3000返210券', '满1200返50券',
       '满2400减150', '满900返40券', '满600返30券', '满6000返660券', '满900减40',
       '满4800减540', '满2600减120', '满200减10', '满2000减100', '满1100减40',
       '满3600返240券', '满2800减140', '满800减30', '满1500返60券', '满1000返40券',
       '满800返30券', '满5400减600', '满1600减80', '满4000减320', '满7200减780',
       '满5100返330券', '满1500减90', '满1000减40', '满1800返70券', '满5200减400',
       '满500减20', '满700返30券', '满3300减210', '满10000减900', '满1400返60券',
       '满2800返140券', '满1600返80券', '满1600减60', '满2600返120券', '满3600返280券',
       '满1200减50', '满7500返650券', '满3000减210', '满2000返100券', '满1500返90券',
       '满3600减240', '满2400返120券', '满2400返160券', '满700减30', '满4000返350券',
       '满2000减200', '满2700减180', '满2400返150券', '满2400减200', '满2200返100券',
       '满1300减50', '满1600返120券', '满1200减60', '满1400返50券', '满2400减120',
       '满3600减180', '满5500减500', '满3900返270券', '满3400减160', '满6600减720

In [7]:
# 对比预热期、活动期、返场期的GMV、净销售额、总订单数、成交订单数、总商品数、成交商品数、客单价、拒退率
def make_tag(day):
    if day < 8:
        return '预热期'
    elif day < 11:
        return '活动期'
    return '返场期'

In [8]:
orders_df['tag'] = orders_df.orderTime.dt.day.apply(make_tag).astype('category')
orders_df['tag'] = orders_df.tag.cat.reorder_categories(['预热期','活动期','返场期'])

In [9]:
temp_df1 = pd.pivot_table(
    orders_df,
    index = 'tag',
    values = ['orderAmount', 'orderID','num', 'userID'],
    aggfunc = {'orderAmount': 'sum',
               'orderID': 'nunique',
               'num': 'sum',
               'userID': 'nunique'}        
).reindex(
       columns = ['orderAmount', 'orderID', 'num', 'userID' ]
).rename(columns={'orderAmount': 'GMV',
                  'orderID':'总订单数',
                  'num': '总商品数',
                  'userID':'总用户数'})

temp_df1

,GMV,总订单数,总商品数,总用户数
tag,,,,
预热期,18004237,6273,17155,6059
活动期,42794651,15441,40789,14769
返场期,31777545,11787,30331,11312


In [10]:
temp_df2 = pd.pivot_table(
    orders_df.query('not chargeback'),
    index = 'tag',
    values = ['payment', 'orderID','num', 'userID'],
    aggfunc = {'payment': 'sum',
               'orderID': 'nunique',
               'num': 'sum',
               'userID': 'nunique'}        
).reindex(
       columns = ['payment', 'orderID', 'num', 'userID' ]
).rename(columns={'payment': '净销售额',
                  'orderID':'成交订单数',
                  'num': '成交商品数',
                  'userID':'成交用户数'})

temp_df2

,净销售额,成交订单数,成交商品数,成交用户数
tag,,,,
预热期,14841296,5164,14159,5006
活动期,25636091,9841,26050,9565
返场期,24215453,9392,24207,9079


In [11]:
result_df = pd.concat( (temp_df1, temp_df2), axis=1)
result_df

,GMV,总订单数,总商品数,总用户数,净销售额,成交订单数,成交商品数,成交用户数
tag,,,,,,,,
预热期,18004237,6273,17155,6059,14841296,5164,14159,5006
活动期,42794651,15441,40789,14769,25636091,9841,26050,9565
返场期,31777545,11787,30331,11312,24215453,9392,24207,9079


In [12]:
result_df['客单价'] = np.round(result_df.净销售额 / result_df.总用户数, 2)
result_df['拒退率'] = 1-result_df.成交订单数 / result_df.总订单数
result_df

,GMV,总订单数,总商品数,总用户数,净销售额,成交订单数,成交商品数,成交用户数,客单价,拒退率
tag,,,,,,,,,,
预热期,18004237,6273,17155,6059,14841296,5164,14159,5006,2449.46,0.176789
活动期,42794651,15441,40789,14769,25636091,9841,26050,9565,1735.80,0.362671
返场期,31777545,11787,30331,11312,24215453,9392,24207,9079,2140.69,0.203190


In [13]:
result_df.to_excel('question1.xlsx')

In [14]:
temp_df3 = pd.pivot_table(
    orders_df.query('not chargeback'),
    index = 'userID',
    values = ['orderID', 'payment','num'],
    aggfunc = {'orderID': 'nunique',
               'payment': 'sum',
               'num': 'sum',
                }        
).rename(columns={'num': '购买商品数',
                  'orderID':'订单数',
                  'payment': '消费金额'
                 }
)

temp_df3['是否购买'] = 1
temp_df3

,购买商品数,订单数,消费金额,是否购买
userID,,,,
user-100000,1,1,1195,1
user-100032,4,2,5074,1
user-100033,3,1,2154,1
user-100034,1,1,1048,1
user-100038,4,1,2616,1
...,...,...,...,...
user-548272,1,1,978,1
user-548274,7,1,9919,1
user-548279,2,1,1848,1


In [15]:
users_df = pd.read_excel('res/user2021.xlsx')
users_df

,userID,user_name,age,phonenum,sex,date,chanelID
0,user-550507,WUe,39,182****7711,女,2021-01-12,渠道-52
1,user-499937,武久,30,177****7858,男,2021-01-08,渠道-89
2,user-515582,周亮l,39,188****6398,女,2021-01-09,渠道-89
3,user-519645,路洋h,34,167****6851,男,2021-01-10,渠道-00
4,user-487799,李宁g,26,148****1375,男,2021-01-03,渠道-52
...,...,...,...,...,...,...,...
69390,user-531290,厉厉,24,193****8590,女,2021-01-08,渠道-52
69391,user-537409,出租尚,33,150****5113,女,2021-01-10,渠道-39
69392,user-511000,农qM,40,171****0753,女,2021-01-09,渠道-98
69393,user-518649,李玉兰,36,186****2539,女,2021-01-10,渠道-00


In [16]:
users_df.drop(index=users_df[(users_df.date.dt.day < 5) |(users_df.date.dt.day > 13 )].index, inplace=True)
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66158 entries, 0 to 69394
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   userID     66158 non-null  object        
 1   user_name  66149 non-null  object        
 2   age        66158 non-null  int64         
 3   phonenum   66158 non-null  object        
 4   sex        66158 non-null  object        
 5   date       66158 non-null  datetime64[ns]
 6   chanelID   66158 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 4.0+ MB


In [17]:
users_df.drop(columns=['phonenum','user_name'],inplace=True)
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66158 entries, 0 to 69394
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   userID    66158 non-null  object        
 1   age       66158 non-null  int64         
 2   sex       66158 non-null  object        
 3   date      66158 non-null  datetime64[ns]
 4   chanelID  66158 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 3.0+ MB


In [18]:
temp_df4 = pd.merge(temp_df3, users_df, how='right', on='userID').fillna(0)


In [19]:
#各个渠道在预热期、活动期、返场期拉新人数、转化率、营收贡献、客单价
(temp_df5 := pd.pivot_table(
    temp_df4,
    index='chanelID',
    values=['订单数','userID','是否购买', '购买商品数', '消费金额'],
    aggfunc={
        '订单数': 'sum',
        'userID': 'count',
        '是否购买': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum'
    }
).applymap(int).rename(columns={'userID': '拉新用户数', '是否购买': '购买用户数'}))

,拉新用户数,购买用户数,消费金额,订单数,购买商品数
chanelID,,,,,
渠道-00,5360,466,1344906,491,1348
渠道-16,2826,232,601559,241,608
渠道-19,2168,186,554238,202,550
渠道-28,4090,389,1072763,410,1065
渠道-39,5663,501,1386851,530,1438
渠道-46,2805,248,643569,260,659
渠道-52,8434,767,2089450,814,2120
渠道-53,7909,731,1990945,764,1972
渠道-56,2841,274,822426,307,784


In [20]:
temp_df5['转化率'] = temp_df5['购买用户数'] / temp_df5['拉新用户数']
temp_df5['ARPU']=temp_df5['消费金额'] / temp_df5['拉新用户数']
temp_df5['ARPPU']=temp_df5['消费金额'] / temp_df5['购买用户数']
temp_df5

,拉新用户数,购买用户数,消费金额,订单数,购买商品数,转化率,ARPU,ARPPU
chanelID,,,,,,,,
渠道-00,5360,466,1344906,491,1348,0.086940,250.915299,2886.064378
渠道-16,2826,232,601559,241,608,0.082095,212.865888,2592.926724
渠道-19,2168,186,554238,202,550,0.085793,255.644834,2979.774194
渠道-28,4090,389,1072763,410,1065,0.095110,262.289242,2757.745501
渠道-39,5663,501,1386851,530,1438,0.088469,244.896874,2768.165669
渠道-46,2805,248,643569,260,659,0.088414,229.436364,2595.036290
渠道-52,8434,767,2089450,814,2120,0.090941,247.741285,2724.185137
渠道-53,7909,731,1990945,764,1972,0.092426,251.731572,2723.590971
渠道-56,2841,274,822426,307,784,0.096445,289.484688,3001.554745


In [21]:
temp_df4['tag'] = temp_df4.date.dt.day.apply(make_tag).astype('category')
temp_df4['tag']=temp_df4.tag.cat.reorder_categories(['预热期','活动期','返场期'])
temp_df4

,userID,购买商品数,订单数,消费金额,是否购买,age,sex,date,chanelID,tag
0,user-550507,0.0,0.0,0.0,0.0,39,女,2021-01-12,渠道-52,返场期
1,user-499937,0.0,0.0,0.0,0.0,30,男,2021-01-08,渠道-89,活动期
2,user-515582,0.0,0.0,0.0,0.0,39,女,2021-01-09,渠道-89,活动期
3,user-519645,0.0,0.0,0.0,0.0,34,男,2021-01-10,渠道-00,活动期
4,user-529356,0.0,0.0,0.0,0.0,24,女,2021-01-08,渠道-19,活动期
...,...,...,...,...,...,...,...,...,...,...
66153,user-531290,0.0,0.0,0.0,0.0,24,女,2021-01-08,渠道-52,活动期
66154,user-537409,0.0,0.0,0.0,0.0,33,女,2021-01-10,渠道-39,活动期
66155,user-511000,0.0,0.0,0.0,0.0,40,女,2021-01-09,渠道-98,活动期
66156,user-518649,0.0,0.0,0.0,0.0,36,女,2021-01-10,渠道-00,活动期


In [22]:
(temp_df6 := pd.pivot_table(
    temp_df4,
    index='chanelID',
    columns='tag',
    values=['订单数','userID','是否购买', '购买商品数', '消费金额'],
    aggfunc={
        '订单数': 'sum',
        'userID': 'count',
        '是否购买': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum'
    }
).applymap(int).rename(columns={'userID': '拉新用户数', '是否购买': '购买用户数'}))

拉新用户数             购买用户数            消费金额                订单数            \
tag        预热期   活动期   返场期   预热期  活动期 返场期    预热期      活动期   返场期 预热期   活动期 返场期   
chanelID                                                                        
渠道-00       90  4724   546     5  461   0   9875  1335031     0   5   486   0   
渠道-16       32  2512   282     2  230   0   4476   597083     0   2   239   0   
渠道-19       38  1904   226     1  185   0   3752   550486     0   1   201   0   
渠道-28       63  3604   423     2  386   1   3276  1066802  2685   2   407   1   
渠道-39      100  4993   570    12  489   0  38871  1347980     0  13   517   0   
渠道-46       41  2512   252     4  244   0   8927   634642     0   4   256   0   
渠道-52      146  7442   846     8  759   0  17262  2072188     0   8   806   0   
渠道-53      123  6990   796     4  727   0   8783  1982162     0   4   760   0   
渠道-56       59  2489   293     5  269   0  14250   808176     0   6   301   0   
渠道-76      126  5476   611    10  583   1  35016  1616678  4108  10   622   1   
渠道-78       42  1980   224     5  201   0  25171   522614     0   5   210   0   
渠道-89      176  9982  1173    12  993   0  34383  2772357     0  13  1046   0   
渠道-98       90  3737   445     7  389   0  14285  1042714     0   7   413   0   

         购买商品数            
tag        预热期   活动期 返场期  
chanelID                  
渠道-00        9  1339   0  
渠道-16        7   601   0  
渠道-19        4   546   0  
渠道-28        4  1058   3  
渠道-39       38  1400   0  
渠道-46        9   650   0  
渠道-52       23  2097   0  
渠道-53       13  1959   0  
渠道-56       14   770   0  
渠道-76       33  1631   4  
渠道-78       18   534   0  
渠道-89       30  2813   0  
渠道-98       15  1066   0

In [23]:
temp_df6.to_excel('qusetion2.xlsx')

In [24]:
(temp_df6['购买用户数'] / temp_df6['拉新用户数'])[['预热期','活动期']]

tag,预热期,活动期
chanelID,,
渠道-00,0.055556,0.097587
渠道-16,0.062500,0.091561
渠道-19,0.026316,0.097164
渠道-28,0.031746,0.107103
渠道-39,0.120000,0.097937
渠道-46,0.097561,0.097134
渠道-52,0.054795,0.101989
渠道-53,0.032520,0.104006
渠道-56,0.084746,0.108076


In [25]:
temp_df4

,userID,购买商品数,订单数,消费金额,是否购买,age,sex,date,chanelID,tag
0,user-550507,0.0,0.0,0.0,0.0,39,女,2021-01-12,渠道-52,返场期
1,user-499937,0.0,0.0,0.0,0.0,30,男,2021-01-08,渠道-89,活动期
2,user-515582,0.0,0.0,0.0,0.0,39,女,2021-01-09,渠道-89,活动期
3,user-519645,0.0,0.0,0.0,0.0,34,男,2021-01-10,渠道-00,活动期
4,user-529356,0.0,0.0,0.0,0.0,24,女,2021-01-08,渠道-19,活动期
...,...,...,...,...,...,...,...,...,...,...
66153,user-531290,0.0,0.0,0.0,0.0,24,女,2021-01-08,渠道-52,活动期
66154,user-537409,0.0,0.0,0.0,0.0,33,女,2021-01-10,渠道-39,活动期
66155,user-511000,0.0,0.0,0.0,0.0,40,女,2021-01-09,渠道-98,活动期
66156,user-518649,0.0,0.0,0.0,0.0,36,女,2021-01-10,渠道-00,活动期


In [26]:
temp_df8 = pd.pivot_table(
    temp_df4,
    index='sex',
    values=['订单数', '购买商品数', '消费金额', '是否购买'],
    aggfunc={
        '订单数': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum',
        '是否购买': 'count'
    }).applymap(int)
temp_df8.to_excel('question3.xlsx')

In [27]:
# 男性还是女性用户对营收贡献更大
temp_df7 = pd.pivot_table(
    temp_df4,
    index='sex',
    values=['订单数', '购买商品数', '消费金额'],
    aggfunc={
        '订单数': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum'
    })
temp_df7

,消费金额,订单数,购买商品数
sex,,,
女,11106698.0,4234.0,11209.0
男,5467335.0,2112.0,5479.0


In [33]:
age_ser = pd.cut(temp_df4.age, bins=[15, 25, 35, 45, 55, 65], right=False)
temp_df4['年龄段'] = age_ser

In [35]:
temp_df4['年龄段'].value_counts().to_excel('年龄段统计.xlsx')

In [29]:
# 那个年龄段的用户对营收贡献更大
temp_df8 = pd.pivot_table(
    temp_df4,
    index=['年龄段', 'sex'],
    values=['订单数', '购买商品数', '消费金额'],
    aggfunc={
        '订单数': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum'
    })
temp_df8.to_excel('qusetion4.xlsx')

In [30]:
temp_df9 = pd.pivot_table(
    temp_df4,
    index=['年龄段'],
    values=['订单数', '购买商品数', '消费金额'],
    aggfunc={
        '订单数': 'sum',
        '购买商品数': 'sum',
        '消费金额': 'sum'
    })
temp_df9.to_excel('qusetion5.xlsx')